# Lecture 21 - Pandas and matplotlib

## Pandas og Matplotlib

### Mål

- Enkel bruk av `pandas`
- Vise ulike måter å visualisere data ved hjelp av `matplotlib`
- Forklare hvordan man bygger opp en plot
- Tips og triks

## Hvorfor er visualisering viktig i naturvitenskap?

* Formidlig
    - formidle resultater på en måte som er lett å forstå slik at forskning blir formidlet på en riktig måte
    - Spesielt viktig for beslutningstakere (statsledere)
* Utforskning
    - visualisering kan bidra til å se på data på en ny måte og gi ny innsikt

## Eksempel - FNs klimarapport (Summary for policymakers)

![](fn_klima.png)

https://www.ipcc.ch/report/ar6/wg1/downloads/report/IPCC_AR6_WGI_SPM_final.pdf

## Eksempel data 

Vi vil bruke offentlig data på Covid19 i Norge (https://github.com/thohan88/covid19-nor-data)

### Hvordan skal vi lese dataene?

Formatet som brukes her heter `csv` (comma-separated-values).

Det finnes en pakke i python som heter `csv`.

Vi kommer til å bruke `pandas` som er et mye brukt bibliotekt for de som driver med "data science"

```
python -m pip install pandas
```

### Åpne dataene

In [ ]:
# Filenavn
path = "infected.csv"
# url = "https://raw.githubusercontent.com/thohan88/covid19-nor-data/37b6b32d32db05b08dda15f002dcc2198836d4c1/data/01_infected/msis/municipality_wide.csv"
url = "https://raw.githubusercontent.com/thohan88/covid19-nor-data/1d53c3ed8b6bc404eef54956e07deaea76c8ef1a/data/01_infected/msis/municipality_wide.csv"


# Download data
# import urllib.request
# urllib.request.urlretrieve(url, path)

import pandas
# df_infected = pandas.read_csv(path)
df_infected = pandas.read_csv(url)

In [ ]:
# Se på info
df_infected.info()

# Se på en beskrivelse
# df_infected.describe()

# Se hvilke kolonner vi har
# df_infected.columns

# Se på de første 10 elementene
# df_infected[:10]
#df_infected.head(n=10)

### Velge noen få kolonner

In [ ]:
# Beolkningstall i hver kommune
kommune_pop = df_infected[["kommune_name", "population"]]
kommune_pop

## Bruke pandas plotting til å plotte populasjonen i hver kommune

In [ ]:
df_infected[:30].plot("kommune_name", "population", "bar", figsize=(10, 6))

## Hva om vi ønsker å se på kun Viken fylke?

La oss se på et enklere problem først. Hvordan kan jeg hente ut elementene som er større enn 10

In [ ]:
import numpy as np
np.random.seed(1)
x = np.random.randint(0, 20, size=20)
x

## Her kan det være fristende å bruke en list comprehension

In [ ]:
[xi for xi in x if xi > 10]

## Her bør vi istedet bruke numpy sin innebygde logikk

In [ ]:
# Konverter arrayet til et bolsk array (True eller False)
larger_than_10 = x > 10
larger_than_10

In [ ]:
# Evaluer x 
x[larger_than_10]

## La oss se hva som er mest effektivt

In [ ]:
x = np.random.randint(0, 20, size=20_000)

In [ ]:
%time y1 = x[x > 10]


In [ ]:
%time y2 = [xi for xi in x if xi > 10]

### Vi kan gjøre samme triks i `pandas`
Hent ut alle rader med `"fylke_name"` lik `"Viken"`

In [ ]:
df_infected["fylke_name"] == "Viken"

In [ ]:
df_viken = df_infected[df_infected["fylke_name"] == "Viken"]
df_viken

In [ ]:
df_infected[df_infected["fylke_name"] == "Viken"].plot("kommune_name", "population", "bar", figsize=(12, 6))

### Se på data for Oslo

Velg data hvor `"fylke_name"` er lik `"Oslo"`

In [ ]:
oslo = df_infected[df_infected['fylke_name'] == "Oslo"]
oslo

Vi ønsker å hente ut en liste over antall smittede. Du kan få verdiene ut ved å bruke `.values`

In [ ]:
oslo_infected = oslo.values[0, 6:].astype(int)
print(oslo_infected[:20])

### Vi kan få data for hele Norge ved å summere kolonnene
Lag et nytt array hvor du summerer over første akse

In [ ]:
df_infected.values.shape

In [ ]:
# Remove colmns with info about fylke, commune etc
all_values = df_infected.values[:, 6:].astype(int)
# Sum along columns
norway_infected = all_values.sum(axis=0)
print(norway_infected[:10])
print(f"Total antall registrert smittede med covid: {norway_infected[-1]:,}")

### Plot dataene

Plot data for Oslo og Norge

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(oslo_infected)
plt.plot(norway_infected)
plt.show()

### Plotte nye smittede
Plot nye smittede ved bruke `np.diff`

In [ ]:
import numpy as np
oslo_new_infected = np.diff(oslo_infected)
norway_new_infected = np.diff(norway_infected)
plt.plot(oslo_new_infected)
plt.plot(norway_new_infected)
plt.savefig("new_infected.png")
plt.show()

### Matplotlib modellen

- Vi skiller mellom "figure" og "axes"
- Vi kan ha flere axes i en figur
- Vi plotter i hver axes

Merk axes er ikke det samme som axis (x-akse og y-akse)

In [ ]:
fig, ax = plt.subplots()
# Vi bruker axes til å plotte
ax.plot(oslo_infected)
ax.plot(norway_infected)
plt.show()
print("ax = ", ax)
print("fig = ", fig)


Hva er foredelen ved å bruke

```python
fig, ax = plt.subplots()
ax.plot(norway_new_infected)
fig.savefig("new_infected.png")
```
mot 

```python
plt.plot(norway_new_infected)
plt.savefig("new_infected.png")
```
?

### Vi har full kontroll over plottene

```python
fig_new_infected, ax_new_infected = plt.subplots()
fig_total_infected, ax_total_infected = plt.subplots()

ax_new_infected.plot(norway_new_infected)
ax_total_infected.plot(norway_infected)

fig_new_infected.savefig("new_infected.png")
fig_total_infected.savefig("total_infected.png")
```

### Hva er det som egentlig skjer når du skriver `plt.plot`?

```python
# Fra pyplot.plot:
def plot(*args, **kwargs):
    ax = gca()  # gca = get current axes
    ax.plot(*args, **kwargs)
```

### Hva er det som egentlig skjer når du skriver `plt.savefig`?

```python
# Fra pyplot.savefig
def savefig(*args, **kwargs):
    fig = gcf()  # gcf = get current figure
    fig.savefig(*args, **kwargs)
```

### Vi kan også bruke dette til å lage subplots

In [ ]:
# plt.subplots(nrows=1, ncols=1)  # default
fig, axs = plt.subplots(2, 1, figsize=(12, 8))

In [ ]:
print(axs)
print(axs.shape)

In [ ]:
print(fig)

### Lage et 2 x 2 plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 4))

In [ ]:
print(axs)
print(axs.shape)


In [ ]:
axs_flat = axs.flatten()
print(axs_flat)
print(axs_flat.shape)

### Here er ulike objekter i en figure

![_](https://matplotlib.org/_images/anatomy.png)

<https://matplotlib.org/_images/anatomy.png>

### I hver axes kan vi lage ulike plot
Plot total anntall smittede i første subplot og nye smittede i andre subplot

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(oslo_infected)
axs[0].plot(norway_infected)

axs[1].plot(oslo_new_infected)
axs[1].plot(norway_new_infected)
plt.show()

### Vi kan også endre organiseringen

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 4))
axs[0].plot(oslo_infected)
axs[0].plot(norway_infected)

axs[1].plot(oslo_new_infected)
axs[1].plot(norway_new_infected)
plt.show()

### I dette tilfelle bruker vi samme x-axse så da kan det være nyttig å dele denne

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
axs[0].plot(oslo_infected)
axs[0].plot(norway_infected)

axs[1].plot(oslo_new_infected)
axs[1].plot(norway_new_infected)
plt.show()

### Vi må sette på en legend som viser hvilke data som er hvilke

In [ ]:
fig, ax = plt.subplots()
l1, = ax.plot(oslo_infected)

l2, = ax.plot(norway_infected)
ax.legend((l1, l2), ("Oslo", "Norway"))
plt.show()

### Det går også an å sende inn labels som argumenter til plots

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.legend()
plt.show()

### Plassering av legend
https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.legend.html

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.legend(loc="best")
plt.show()

Hvis du ønsker å plassere legend utenfor kan du bruke `bbox_to_anchor`

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.legend(bbox_to_anchor=(0, 1), loc='upper left')
plt.show()

### Hva med legend for subplots?

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
axs[0].plot(oslo_infected, label="Oslo")
axs[0].plot(norway_infected, label="Norway")

axs[1].plot(oslo_new_infected, label="Oslo")
axs[1].plot(norway_new_infected, label="Norway")

for ax in axs:
    ax.legend()
plt.show()

### Hva om vi ønsker en felles lenged?

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
l1, = axs[0].plot(oslo_infected, color="tab:blue")
l2, = axs[0].plot(norway_infected, color="tab:orange")

axs[1].plot(oslo_new_infected, color="tab:blue")
axs[1].plot(norway_new_infected, color="tab:orange")

fig.subplots_adjust(right=0.88)
fig.legend((l1, l2), ("Oslo", "Norway"), loc='center right')
# fig.savefig("subplots_common_legend.png")

# Kan også gjøre dette, men da kan legenden bli kuttet bort ved lagring
# axs[1].legend((l1, l2), ("Oslo", "Norway"), bbox_to_anchor=(1, 0.5), loc='center left')
# Send med bbox_inches="tight" for å ungå dette
# fig.savefig("subplots_common_legend.png") #, bbox_inches="tight")
# plt.show()
# !open subplots_common_legend.png

### Sette på grid

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

### Sette på tittel

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.set_title("Total infected")
#plt.title("Title")  # sjekk plt.title??
print(ax.title)
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

### Xlabels og Ylabels

In [ ]:
fig, ax = plt.subplots()
ax.plot(oslo_infected, label="Oslo")
ax.plot(norway_infected, label="Norway")
ax.grid()
ax.set_xlabel("Days")
ax.set_ylabel("Total infected")
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.show()

### Xticks

In [ ]:
fig, ax = plt.subplots()
x = np.arange(len(oslo_infected))
ax.plot(x, oslo_infected, label="Oslo")
ax.plot(x, norway_infected, label="Norway")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xticks([0, 150, 200, 352, 420, 500])
plt.show()

### Yticks

In [ ]:
fig, ax = plt.subplots()
x = np.arange(len(oslo_infected))
ax.plot(x, oslo_infected, label="Oslo")
ax.plot(x, norway_infected, label="Norway")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_yticks([0, 50_000, 100_000, 150_000])
ax.set_xticks([0, 150, 200, 352, 420, 500])
plt.show()

### Xticklabels

In [ ]:
fig, ax = plt.subplots()
x = np.arange(50, len(oslo_infected)+50)
ax.plot(x, oslo_infected, label="Oslo")
ax.plot(x, norway_infected, label="Norway")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xticks([75, 175])
ax.set_xticklabels(["Point1", "Point2"])
plt.show()

## Vi ønsker å ha datoene på x-aksen

Disse ligger i kolonne navnene

In [ ]:
dates = df_infected.columns[6:].to_numpy()
print(dates)

## Bruk datoene some xticklabels

In [ ]:
fig, ax = plt.subplots()
# Få datoene fra dataene
dates = np.array(df_infected.keys()[6:])

x = np.arange(len(oslo_infected))
# Velg xticks
xticks = np.linspace(0, len(oslo_infected)-1, 6).astype(int)
ax.plot(x, oslo_infected, label="Oslo")
ax.plot(x, norway_infected, label="Norway")
ax.grid()
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xticks(xticks)
ax.set_xticklabels(dates[xticks], rotation=30)
ax.set_xlabel("Date")
ax.set_ylabel("Total infected")
plt.show()

### Hvordan dele aksene mellom subplots?

Plot både totalt antall smittede og nye smittede med datoer som xticklabels

In [ ]:
fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 8))
# Få datoene fra dataene
dates = np.array(df_infected.keys()[6:])

x = np.arange(len(oslo_infected))

xticks =  np.linspace(0, len(oslo_infected)-1, 6).astype(int)
l1, = axs[0].plot(x, oslo_infected)
l2, = axs[0].plot(x, norway_infected)
axs[0].set_ylabel("Total infected")

axs[1].plot(x[:-1], oslo_new_infected)
axs[1].plot(x[:-1], norway_new_infected)
axs[1].set_ylabel("Total new infected")

for ax in axs:
    ax.grid()
    ax.set_xticks(xticks)
    
# Hvis du har 2, 2 - bruk axs.flatten()!
axs[1].set_xticklabels(dates[xticks], rotation=30)
axs[1].set_xlabel("Date")

fig.legend((l1, l2), ("Oslo", "Norway"), bbox_to_anchor=(0.5, 0.5), loc='center', ncol=2)
plt.show()

## Style

In [ ]:
print(plt.style.available)

In [ ]:
plt.style.use('seaborn')
fig, ax = plt.subplots()
# Vi bruker axes til å plotte
ax.plot(oslo_infected)
ax.plot(norway_infected)
plt.show()

### Seaborn er en populær pakke som bygger på matplotlib

Du kan installere denne med `pip`: 
```
python3 -m pip install seaborn
```
https://seaborn.pydata.org/index.html

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")

df = sns.load_dataset("penguins")
sns.pairplot(df, hue="species")

### XKCD style

In [ ]:
plt.style.use('default')
with plt.xkcd():
    fig, ax = plt.subplots()
    # Vi bruker axes til å plotte
    ax.plot(oslo_infected)
    ax.plot(norway_infected)
    plt.show()

### Styling lines

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
axs[0].plot(oslo_infected[:20], linestyle="--", color="k")
axs[0].plot(norway_infected[:20], linestyle="", marker=".", color="r", markersize=10)

axs[1].plot(oslo_new_infected[:20], linestyle=":", marker="*", markersize=3, color="b")
axs[1].plot(norway_new_infected[:20], linestyle="-", linewidth="3", color="g")
plt.show()

### Hvordan variere marker og farge i en loop?

In [ ]:
rogaland = df_infected[df_infected["fylke_name"] == "Rogaland"]


In [ ]:
from itertools import cycle
colors = cycle(["r", "b", "g", "y", "c", "m"])
markers = cycle(["x", "v", "o", "<", "."])
fig, ax = plt.subplots(figsize=(10, 10))
for x, marker, color in zip(rogaland.values, markers, colors):
    name = x[2]
    v = x[6:]
    ax.plot(v[::20], label=name, marker=marker, color=color)
    
ax.legend()

### Hvilket fargekart skal vi velge?

Det er nok å velge i mellom: <https://matplotlib.org/stable/tutorials/colors/colormaps.html>


In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("xAoljeRJ3lU")

### Riktig fargevalg er viktig når man skal visualisere 2D data

In [ ]:
import scipy.misc
img = scipy.misc.ascent()
print(img)
print(img.shape)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
plt.show()

### En mye brukt colormap er `jet` - men IKKE bruk denne til å visualisere data!

Den lager for mye kontrast og du kan fort tro at det er ting i bildet som egentlig ikke er der.
Men bruk gjerne denne om du bare ønsker en colormap med masse farger :)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img, cmap="jet")
plt.show()

### Plotte smitte i dag basert på fylker

In [ ]:
## Put alle rader som hører til i samme fylke sammen
fylker = df_infected.groupby("fylke_name")

In [ ]:
# Summer all bidragene inad i hver gruppe
fylker_sum = fylker.sum()
fylker_sum

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(fylker))
ax.bar(x, fylker_sum.values[:, -1])
ax.set_xticks(x)
ax.set_xticklabels(fylker.groups.keys(), rotation=30)
ax.set_title(f"Total infected on {fylker_sum.keys()[-1]}")
ax.set_xlabel("Fylke")
ax.set_ylabel("Total smittet")
ax.grid()
plt.show()

### Hva om vi ønsker et bar plot med to dager?

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(fylker))
width = 0.4
n = 365
ax.bar(x + 0.2, fylker_sum.values[:, -1], label=fylker_sum.keys()[-1], width=width)
ax.bar(x - 0.2, fylker_sum.values[:, -n], label=fylker_sum.keys()[-n], width=width)

ax.set_xticks(x)
ax.set_xticklabels(fylker.groups.keys(), rotation=30)
ax.set_xlabel("Fylke")
ax.set_ylabel("Total smittet")
ax.set_title("Total smitte for to dager")
ax.legend()
plt.show()

### Hva om vi ønsker et bar plot med 5 siste måneder?

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(fylker))
num = 5
days_in_month = 30
width = 1/(num + 1)

for i in range(5):
    ax.bar(
        x - i*width, 
        fylker_sum.values[:, -(days_in_month*i+1)], 
        label=fylker_sum.keys()[-(days_in_month*i+1)], 
        width=width
    )


ax.set_xticks(x - width*num/2)
ax.set_xticklabels(fylker.groups.keys(), rotation=30)
ax.set_xlabel("Fylke")
ax.set_ylabel("Total smittet")
ax.set_title("Total smitte siste 5 måneder")
ax.legend()
plt.show()

### Pie chart

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))


values = fylker_sum.values[:, -1]
labels = np.array(list(fylker.groups.keys()))

ax.pie(values, labels=labels)
ax.axis("equal")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

values = fylker_sum.values[:, -1]
labels = np.array(list(fylker.groups.keys()))

explode = np.zeros(len(values))
explode[labels == "Oslo"] = 0.1

ax.pie(values, explode=explode, labels=labels,  autopct="%1.1f%%", shadow=True)
ax.axis("equal")
plt.show()

### Hvordan lære mer?

Sjekk ut Matplotlib gallery: https://matplotlib.org/stable/gallery/index.html

Finn ett plot som ligner på det du ønsker å plotte og endre litt på det.


### Andre plottebilioteker i python

![_](https://optimise2.assets-servd.host/voracious-blesbok/production/Blog/PythonVisLandscape.jpg?w=1200&auto=compress%2Cformat&fit=crop&dm=1632326979&s=35cf543e04fd14bcc881ef8e70363860)
https://www.anaconda.com/blog/python-data-visualization-2018-why-so-many-libraries

https://youtu.be/FytuB8nFHPQ